<a href="https://colab.research.google.com/github/Zinwaiyan274/Twitter-sentiment-analysis-using-RNN-and-GloVe-embedding/blob/main/Final_Testing_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  import numpy as np # linear algebraimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer


#essential libs
import pandas as pd
import numpy as np


from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
headers=['ID','Entity','label','Feature']

train_data = pd.read_csv("https://raw.githubusercontent.com/Zinwaiyan274/Twitter-Sentiment-Analysis/main/twitter_training.csv", sep=',', names=headers)
val_data = pd.read_csv("https://raw.githubusercontent.com/Zinwaiyan274/Twitter-Sentiment-Analysis/main/twitter_validation.csv", sep=',', names=headers)

df = train_data.append(val_data)
# # df = train_data
# df
# # train data = df

# # for train_df data

df["label"] = df["label"].replace("Positive" , int("2"))
df["label"] = df["label"].replace("Negative", int("1"))
df["label"] = df["label"].replace("Neutral", int("0"))
# df = df[df["label"].str.contains("Irrelevant") == False]
df.drop(df.loc[df['label']== "Irrelevant"].index, inplace=True)
df.dropna()

,ID,Entity,label,Feature
1,2401,Borderlands,2,I am coming to the borders and I will kill you...
2,2401,Borderlands,2,im getting on borderlands and i will kill you ...
3,2401,Borderlands,2,im coming on borderlands and i will murder you...
4,2401,Borderlands,2,im getting on borderlands 2 and i will murder ...
5,2401,Borderlands,2,im getting into borderlands and i can murder y...
...,...,...,...,...
993,314,Amazon,1,Please explain how this is possible! How can t...
994,9701,PlayStation5(PS5),2,Good on Sony. As much as I want to see the new...
997,2652,Borderlands,2,Today sucked so it’s time to drink wine n play...
998,8069,Microsoft,2,Bought a fraction of Microsoft today. Small wins.


In [ ]:
val_data["label"] = val_data["label"].replace("Positive" , int("2"))
val_data["label"] = val_data["label"].replace("Negative", int("1"))
val_data["label"] = val_data["label"].replace("Neutral", int("0"))
val_data.drop(val_data.loc[val_data['label']== "Irrelevant"].index, inplace=True)




val_label = val_data.label
val_Feature = val_data.Feature
val_Feature.shape, val_label.shape
val_label



1      0
2      1
3      1
4      0
5      1
      ..
993    1
994    2
997    2
998    2
999    0
Name: label, Length: 828, dtype: object

In [ ]:
val_Feature

1      BBC News - Amazon boss Jeff Bezos rejects clai...
2      @Microsoft Why do I pay for WORD when it funct...
3      CSGO matchmaking is so full of closet hacking,...
4      Now the President is slapping Americans in the...
5      Hi @EAHelp I’ve had Madeleine McCann in my cel...
                             ...                        
993    Please explain how this is possible! How can t...
994    Good on Sony. As much as I want to see the new...
997    Today sucked so it’s time to drink wine n play...
998    Bought a fraction of Microsoft today. Small wins.
999    Johnson & Johnson to stop selling talc baby po...
Name: Feature, Length: 828, dtype: object

In [ ]:
df.Feature

1      I am coming to the borders and I will kill you...
2      im getting on borderlands and i will kill you ...
3      im coming on borderlands and i will murder you...
4      im getting on borderlands 2 and i will murder ...
5      im getting into borderlands and i can murder y...
                             ...                        
993    Please explain how this is possible! How can t...
994    Good on Sony. As much as I want to see the new...
997    Today sucked so it’s time to drink wine n play...
998    Bought a fraction of Microsoft today. Small wins.
999    Johnson & Johnson to stop selling talc baby po...
Name: Feature, Length: 62284, dtype: object

In [ ]:
df["label"]

1      2
2      2
3      2
4      2
5      2
      ..
993    1
994    2
997    2
998    2
999    0
Name: label, Length: 62284, dtype: object

In [ ]:
len(df.Feature), len(df["label"])

(62284, 62284)

In [ ]:
# X = df.Feature

# df['label']
Y = df['label']
Y = pd.get_dummies(Y)
# Y = Y.iloc[:20401] 
Y

,0,1,2
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
5,0,0,1
...,...,...,...
993,0,1,0
994,0,0,1
997,0,0,1
998,0,0,1


In [ ]:
import re
REPLACE_WITH_SPACE = re.compile("(@)")
SPACE = " "



# How about stop words??

import nltk
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean(text):  
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text



from nltk.parse.transitionparser import remove



#remove link
def remove_links(tweet):
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet



# # df['Feature']= df["Feature"].apply(preprocess_reviews)
# df['Feature']= df["Feature"].apply(clean)
# # df['Feature']= df["Feature"].apply(preprocess)
# df['Feature']= df["Feature"].apply(remove_links)



#remove twitter user ( acc username)
def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet


# df['Feature']= df["Feature"].apply(remove_users)



#remove punctuation
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'
def preprocess(sent):
    sent = remove_users(sent)
    sent = remove_links(sent)
    sent = sent.lower()
    sent = re.sub('['+my_punctuation + ']+', ' ', sent) # strip punctuation
    sent = re.sub('\s+', ' ', sent) #remove double spacing
    sent = re.sub('([0-9]+)', '', sent) # remove numbers
    sent_token_list = [word for word in sent.split(' ')]
    sent = ' '.join(sent_token_list)
    return sent

# df['Feature']= df["Feature"].apply(preprocess)



# remove emoticons


!pip install emoji
import emoji
import regex as re
# import renew_tweet
import sys
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


# df['Feature']= df["Feature"].apply(remove_emoji)

import re
from nltk.stem.porter import *
from nltk.corpus import stopwords

def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''
    
    # convert to lowercase
    text = tweet.lower()
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

# df['Feature']= df["Feature"].apply(tweet_to_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
X_train, X_test, Y_train, Y_test = train_test_split(df["Feature"],Y, test_size = 0.33, random_state = 42)
X_test, X_val, y_test, y_val = train_test_split(X_train,Y_train, train_size = 0.75)
#train


X_train





7506     used cheats on my smurf account. did an overwa...
26601      I have NEVER played an Apollo Creed Game... but
32086    dont forget and tpci is partnering with t * h ...
28317    Thank u @excalibagel uwu.. 4 fave games.. -Hal...
24597          fuck you google! pic.twitter.com/wo7ntZsGNL
                               ...                        
66808    Richard Johnson & Martin Johnson working on va...
47221    Me Too by Meghan Trainor is set in this Home D...
1139                                    then want to thank
19452                           LOVE my Warcraft fam!!! . 
69271    That sucks. If you say that CD Project Red can...
Name: Feature, Length: 41730, dtype: object

In [ ]:
import pickle

# loading
with open('/content/drive/MyDrive/TSA_pj/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

import tensorflow
from tensorflow.keras.models import load_model
model_file = "/content/drive/MyDrive/TSA_pj/1st_model (1).h5"
model = tensorflow.keras.models.load_model(model_file)

#Testing

In [ ]:
F =  df["Feature"]
# F = F.head()
F = F.to_list()
# X = F.tolist()
F


['I am coming to the borders and I will kill you all,',
 'im getting on borderlands and i will kill you all,',
 'im coming on borderlands and i will murder you all,',
 'im getting on borderlands 2 and i will murder you me all,',
 'im getting into borderlands and i can murder you all,',
 "So I spent a few hours making something for fun. . . If you don't know I am a HUGE @Borderlands fan and Maya is one of my favorite characters. So I decided to make myself a wallpaper for my PC. . Here is the original image versus the creation I made :) Enjoy! pic.twitter.com/mLsI5wf9Jg",
 "So I spent a couple of hours doing something for fun... If you don't know that I'm a huge @ Borderlands fan and Maya is one of my favorite characters, I decided to make a wallpaper for my PC.. Here's the original picture compared to the creation I made:) Have fun! pic.twitter.com / mLsI5wf9Jg",
 "So I spent a few hours doing something for fun... If you don't know I'm a HUGE @ Borderlands fan and Maya is one of my fav

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import pickle
# tokenizer = Tokenizer(num_words=max_fatures, split=' ')

"""

"war" itself alone is Netural. that makes sense enough, noting wrong that much.  
"terrible war" is turely Negative  beace if the word of "terrible".


"How are you" cause error


"""
# user = str(input(" Type smt to test the model"))
# user = X_test[:99]

# user = (" ugly face")
# user = (" terrible")
# user = ("Angry")


# user = ("Terrible war")
# user = (" walking")
user = (" Mom is comming")
# user = (" how is your day going?")
# user = ("Black sheep")


# user = (" love")
# user = (" Beautiful colorful dress ")
# user = ("clever trick")

predict_label = []

# user = [" how is your day going?" , "Black sheep", "Black sheep"]  

def label_encode(output):
    if output == "Netural":
      # print(1)
      output = 1
    elif output == "Postivie":
      # print(2)
      output = 2
       
    elif output == "Negitive":
      # print(0)  
      output = 0

    
    predict_label.append(output)
       
    print(output)
  


def model_pip(user):
  # return user
  prediction = model.predict(user)
  # print(prediction)
  predicted = np.argmax(prediction)
  
  # predicted = np.argmax(prediction) 
  # print(predicted)

  if predicted == 1:
    # print("Netural")
    output = "Netural"

  elif (predicted) == 2:
    # print("Postivie")
    output = "Postivie"

  elif (predicted) == 0:
    # print("Negitive")
    output = "Negitive"

  else:
    # print("Netural")
    output = "Netural"

  # print(output) 
  label_encode(output)

   

def token(user):
  user= tweet_to_words(user)
  user = tokenizer.texts_to_sequences(user)
  user = pad_sequences(user)
  # print(user)
  model_pip(user)
  # print(user)

def main(user):
  user  = remove_links(user)
  user  = remove_emoji(user)
  user = preprocess(user)
  user  =preprocess(user)
  # user  =remove_users(remove_users)
  user  =clean(user)
  # return user
  # user = tokenizer.texts_to_sequences(user)
  # return user
     
  token(user)
  # token(user)

# for i in user:
#   main(i)

from tqdm import tqdm
for sentence in tqdm(val_Feature):
  test =  main(sentence)

type(predict_label)

  
# main(user)
 
# user = (" how is your day going?")
# user = ("Black sheep")


# user = (" love")




  0%|          | 0/828 [00:00<?, ?it/s]WARNING:tensorflow:Model was constructed with shape (None, 163) for input KerasTensor(type_spec=TensorSpec(shape=(None, 163), dtype=tf.float32, name='embedding_3_input'), name='embedding_3_input', description="created by layer 'embedding_3_input'"), but it was called on an input with incompatible shape (None, 1).


1/1 [==============================] - 1s 1s/step


  0%|          | 1/828 [00:01<14:59,  1.09s/it]

1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 19ms/step


  0%|          | 3/828 [00:01<04:35,  2.99it/s]

...

100%|█████████▉| 826/828 [02:00<00:00, 10.52it/s]

1
1/1 [==============================] - 0s 23ms/step
1
1/1 [==============================] - 0s 279ms/step


100%|██████████| 828/828 [02:01<00:00,  6.84it/s]

1


list

In [ ]:
predict_label

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
# val_label = val_label.to_numpy()
val_label

1      0
2      1
3      1
4      0
5      1
      ..
993    1
994    2
997    2
998    2
999    0
Name: label, Length: 828, dtype: object

In [ ]:
predict_label =np.array(predict_label)
type(predict_label) 
# final_label.values

numpy.ndarray

In [ ]:
predict_label

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(list(val_label), predict_label))
       

              precision    recall  f1-score   support

           0       0.16      0.02      0.04       285
           1       0.32      0.89      0.47       266
           2       0.82      0.13      0.22       277

    accuracy                           0.34       828
   macro avg       0.43      0.35      0.24       828
weighted avg       0.43      0.34      0.24       828



In [ ]:
F =  df["Feature"]
F

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from tqdm import tqdm
predict_label = []
for sentence in tqdm(F.to_list()):
  test =  main(sentence)
  predict_label.append(test)

predict_label

In [ ]:
from tqdm import tqdm

user = []

for i in tqdm(df["Feature"]):
  i.append[user.apply|()]

main(user)


In [ ]:
df.label[:5]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predict_label, df.label[5]))
       

In [ ]:
val_data["label"]

In [ ]:
val_data["Feature"]

In [ ]:
len(val_data["label"]), len(val_data["Feature"])

In [ ]:
print("Accuracy of ELMO is:",accuracy_score(y_test,y_preds))


In [ ]:
val_data.Feature

In [ ]:
len(val_data.Feature), len(val_data.label)

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_val, y_val)
# print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
# print("Generate predictions for 3 samples")
# predictions = model.predict(val_data.Feature[:3])
# print("predictions shape:", predictions.shape)

In [ ]:
result = model.evaluate(val_Feature)

In [ ]:
# df['Feature']= df["Feature"].apply(tweet_to_words)
# df['Feature']= df["Feature"].apply(remove_emoji)
# df['Feature']= df["Feature"].apply(preprocess)
# df['Feature']= df["Feature"].apply(remove_users)
# df['Feature']= df["Feature"].apply(remove_links)
# df['Feature']= df["Feature"].apply(clean)

# user= user.apply(tweet_to_words)
# user= user.apply(remove_emoji)
# user= user.apply(preprocess)
# user= user.apply(remove_users)
# user= user.apply(remove_links)
# user= user.apply(clean)




